import numpy as np
import pandas as pd

from statsmodels.miscmodels.ordinal_model import OrderedModel
pd.set_option('display.max_columns', None)

In [5]:
import numpy as np
import pandas as pd

from statsmodels.miscmodels.ordinal_model import OrderedModel
pd.set_option('display.max_columns', None)


df = pd.read_csv('ENEM_MESCLADOS.csv').drop(columns = ['Unnamed: 0','Unnamed: 0.1']).dropna()
df["hip_nula"]=0
df

,NU_IDADE,TP_SEXO,RECURSO_ESPECIAL,Cor_Branca,Cor_Preta,Cor_Parda,Cor_Amarela,Cor_Indígena,ESC_PUBL,CAT_CH,CAT_CN,CAT_LC,CAT_MT,CAT_REDACAO,Estudo_Pai,Estudo_Mae,Ocupacao_Pai_1,Ocupacao_Pai_2,Ocupacao_Pai_3,Ocupacao_Pai_4,Ocupacao_Pai_5,Ocupacao_Mae_1,Ocupacao_Mae_2,Ocupacao_Mae_3,Ocupacao_Mae_4,Ocupacao_Mae_5,Pessoas_casa,Renda,AFD_1,AFD_2,AFD_3,AFD_4,ATU,HAD,hip_nula
0,0.235294,1,1,1,0,0,0,0,0,3,3,3,2,3,0.000000,0.666667,1,0,0,0,0,0,0,0,1,0,0.105263,0.176471,0.595,0.000,0.243,0.081,0.241206,0.34375,0
1,0.352941,1,0,0,0,1,0,0,1,2,2,2,2,2,0.166667,0.166667,0,1,0,0,0,0,1,0,0,0,0.157895,0.117647,0.931,0.000,0.000,0.069,0.175879,0.25000,0
2,0.220588,1,1,0,0,1,0,0,1,2,2,3,2,2,0.666667,0.666667,0,1,0,0,0,0,1,0,0,0,0.421053,0.176471,0.980,0.001,0.019,0.000,0.169347,0.27500,0
3,0.235294,1,1,0,0,1,0,0,1,2,2,3,2,2,0.666667,0.666667,0,1,0,0,0,0,1,0,0,0,0.105263,0.176471,0.767,0.000,0.233,0.000,0.097990,0.30000,0
4,0.220588,0,1,0,0,1,0,0,1,3,2,3,3,3,0.500000,0.166667,1,0,0,0,0,1,0,0,0,0,0.105263,0.000000,0.756,0.000,0.225,0.019,0.140704,0.26875,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881725,0.220588,1,1,1,0,0,0,0,1,2,3,3,3,2,0.500000,0.666667,0,0,0,1,0,0,0,0,1,0,0.210526,0.117647,0.712,0.000,0.258,0.000,0.182412,0.27500,0
881726,0.235294,1,1,1,0,0,0,0,1,3,2,3,2,3,0.833333,0.833333,0,0,0,0,1,0,0,0,0,1,0.105263,0.647059,0.763,0.031,0.126,0.080,0.181910,0.26250,0
881727,0.235294,1,1,0,1,0,0,0,1,3,2,3,2,3,0.833333,1.000000,0,0,0,1,0,0,0,0,1,0,0.210526,0.470588,0.503,0.155,0.077,0.265,0.171859,0.47500,0
881728,0.235294,1,1,0,0,1,0,0,1,3,2,3,3,3,0.666667,1.000000,0,0,1,0,0,0,0,0,1,0,0.157895,0.647059,0.395,0.474,0.131,0.000,0.140704,0.60000,0


In [6]:
def reg_log (x,y,):
    mod_log = OrderedModel(y, X, distr='logit')
    res_log = mod_log.fit(method='bfgs', disp=False)
    col =['Var','coef','std','z','p-value','2,5%','97,5%']
    resul = pd.DataFrame(res_log.summary().tables[1][1:],columns = col)
    params = res_log.params
    pvalues = res_log.pvalues
    
    return [resul,params,pvalues]

def cria_exogenas(DATA,RemoCol):
    exo= DATA[list (DATA.drop(columns = RemoCol).columns)]
    return exo

In [7]:
%%time
# Declaração da variável endógena
Var_Dep ='CAT_MT'

# Lista iniciais de variáveis a serem desconcideradas
Var_Remo = ['CAT_MT', 'CAT_CH','CAT_CN','CAT_LC','CAT_REDACAO','Cor_Indígena','RECURSO_ESPECIAL','NU_IDADE']

# Critério para p-value
crit =0.05

y = df[Var_Dep]
X = cria_exogenas(df,Var_Remo)

[resul,parm,pval]=reg_log(X,y)

print(resul)

extrai = pval[0:-4][pval>=crit].index

for i in extrai:
    Var_Remo.append(i)

conta=0
    
while (len(extrai)!=0):
    conta=conta+1
    print("loop: ",conta)
    extrai=[]
    X = cria_exogenas(df,Var_Remo)
    [resul,parm,pval]=reg_log(X,y)
    extrai = pval[0:-4][pval>=crit].index
    for i in extrai:
        Var_Remo.append(i)
        
    print(resul)


               Var        coef        std          z p-value       2,5%  \
0          TP_SEXO     -0.6379      0.005   -130.203   0.000     -0.648   
1       Cor_Branca      0.3043      0.016     19.544   0.000      0.274   
2        Cor_Preta     -0.2206      0.017    -13.211   0.000     -0.253   
3        Cor_Parda     -0.0528      0.015     -3.422   0.001     -0.083   
4      Cor_Amarela      0.1469      0.022      6.632   0.000      0.103   
5         ESC_PUBL     -0.9556      0.008   -116.065   0.000     -0.972   
6       Estudo_Pai      0.3451      0.011     30.889   0.000      0.323   
7       Estudo_Mae      0.5191      0.012     42.238   0.000      0.495   
8   Ocupacao_Pai_1      0.0671      0.011      6.132   0.000      0.046   
9   Ocupacao_Pai_2      0.0367      0.009      3.888   0.000      0.018   
10  Ocupacao_Pai_3      0.1669      0.009     18.332   0.000      0.149   
11  Ocupacao_Pai_4      0.3353      0.010     33.067   0.000      0.315   
12  Ocupacao_Pai_5      0

In [58]:
resul.to_csv('gambiarra.csv',index=False)

In [59]:
resultado = pd.read_csv('gambiarra.csv', index_col=False)

In [60]:
resultado.sort_values(['coef'])

,Var,coef,std,z,p-value,"2,5%","97,5%"
26,0/1,-8.3596,0.115,-73.006,0.000,-8.584,-8.135
18,Pessoas_casa,-1.3661,0.035,-39.529,0.000,-1.434,-1.298
5,ESC_PUBL,-0.9556,0.008,-116.066,0.000,-0.972,-0.940
0,TP_SEXO,-0.6379,0.005,-130.203,0.000,-0.648,-0.628
22,AFD_3,-0.4898,0.051,-9.614,0.000,-0.590,-0.390
2,Cor_Preta,-0.2206,0.017,-13.217,0.000,-0.253,-0.188
3,Cor_Parda,-0.0528,0.015,-3.424,0.001,-0.083,-0.023
9,Ocupacao_Pai_2,0.0367,0.009,3.888,0.000,0.018,0.055
13,Ocupacao_Mae_1,0.0653,0.012,5.338,0.000,0.041,0.089
8,Ocupacao_Pai_1,0.0671,0.011,6.131,0.000,0.046,0.089


In [49]:
pd.read_csv('gambiarra.csv')

,Unnamed: 0,Var,coef,std,z,p-value,"2,5%","97,5%"
0,0,TP_SEXO,-0.6379,0.005,-130.203,0.000,-0.648,-0.628
1,1,Cor_Branca,0.3043,0.016,19.552,0.000,0.274,0.335
2,2,Cor_Preta,-0.2206,0.017,-13.217,0.000,-0.253,-0.188
3,3,Cor_Parda,-0.0528,0.015,-3.424,0.001,-0.083,-0.023
4,4,Cor_Amarela,0.1469,0.022,6.634,0.000,0.104,0.190
5,5,ESC_PUBL,-0.9556,0.008,-116.066,0.000,-0.972,-0.940
6,6,Estudo_Pai,0.3451,0.011,30.889,0.000,0.323,0.367
7,7,Estudo_Mae,0.5191,0.012,42.239,0.000,0.495,0.543
8,8,Ocupacao_Pai_1,0.0671,0.011,6.131,0.000,0.046,0.089
9,9,Ocupacao_Pai_2,0.0367,0.009,3.888,0.000,0.018,0.055


In [61]:
resultado['coef_modulo']=resultado['coef'].apply(lambda x: abs(x))

In [68]:
resultado.iloc[0:-4].sort_values('coef_modulo',ascending=False)

,Var,coef,std,z,p-value,"2,5%","97,5%",coef_modulo
19,Renda,2.3957,0.017,141.381,0.000,2.363,2.429,2.3957
25,HAD,2.2540,0.022,103.793,0.000,2.211,2.297,2.2540
18,Pessoas_casa,-1.3661,0.035,-39.529,0.000,-1.434,-1.298,1.3661
23,AFD_4,1.1725,0.057,20.470,0.000,1.060,1.285,1.1725
21,AFD_2,0.9768,0.059,16.624,0.000,0.862,1.092,0.9768
5,ESC_PUBL,-0.9556,0.008,-116.066,0.000,-0.972,-0.940,0.9556
24,ATU,0.9401,0.053,17.676,0.000,0.836,1.044,0.9401
0,TP_SEXO,-0.6379,0.005,-130.203,0.000,-0.648,-0.628,0.6379
20,AFD_1,0.5241,0.048,10.860,0.000,0.430,0.619,0.5241
7,Estudo_Mae,0.5191,0.012,42.239,0.000,0.495,0.543,0.5191


In [71]:
resultado.iloc[0:-4][['Var','coef','std','z','p-value']]

,Var,coef,std,z,p-value
0,TP_SEXO,-0.6379,0.005,-130.203,0.000
1,Cor_Branca,0.3043,0.016,19.552,0.000
2,Cor_Preta,-0.2206,0.017,-13.217,0.000
3,Cor_Parda,-0.0528,0.015,-3.424,0.001
4,Cor_Amarela,0.1469,0.022,6.634,0.000
5,ESC_PUBL,-0.9556,0.008,-116.066,0.000
6,Estudo_Pai,0.3451,0.011,30.889,0.000
7,Estudo_Mae,0.5191,0.012,42.239,0.000
8,Ocupacao_Pai_1,0.0671,0.011,6.131,0.000
9,Ocupacao_Pai_2,0.0367,0.009,3.888,0.000


SyntaxError: invalid syntax (2725834251.py, line 1)